In [3]:
# Passo 1: Importar bibliotecas necessárias
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras import optimizers
from sklearn import model_selection
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import keras
from tensorflow.keras import regularizers
# importando o ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import HeNormal, HeUniform
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import tensorflow as tf
import cv2


: 

In [4]:
# Passo 2: Carregar o dataset CIFAR-10
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.cifar10.load_data()


# Passo 3: Função para redimensionar as imagens usando OpenCV (cv2)
def resize_with_cv2(images, new_size):
    resized_images = []
    for img in images:
        resized_img = cv2.resize(img, new_size)
        resized_images.append(resized_img)
    return np.array(resized_images)

# Passando os dados para float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Dividindo os dados por 255.0 para normalizar
X_train = X_train / 255.0

# Transformando os labels em one-hot-encoding
Y_train_one_hot = keras.utils.to_categorical(Y_train, 10)
Y_test_one_hot = keras.utils.to_categorical(Y_test, 10)

# Passo 4: Redimensionar as imagens para 224x224x3
new_size = (224, 224)
X_train_resized = resize_with_cv2(X_train, new_size)
X_test_resized = resize_with_cv2(X_test, new_size)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


In [ ]:
BATCH_SIZE = 32
RANDOM_STATE = 42

In [ ]:
from tensorflow.keras.applications import VGG16

# Carregar o modelo VGG16 pré-treinado no ImageNet
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Visualizar a arquitetura
base_model.summary()

# Congelar as camadas convolucionais
base_model.trainable = False

# Criando o modelo completo
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=10, activation='softmax'))

# Criando checkpoints
mc = ModelCheckpoint('best_model2.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
# Compilando o modelo
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train_one_hot, epochs=10, batch_size=BATCH_SIZE, validation_data=(X_test, Y_test_one_hot), verbose=1, callbacks=[mc])

In [ ]:
model = load_model('best_model2.keras')

In [ ]:
# Calculando a acuracia
_, accuracy_train = model.evaluate(X_train, Y_train_one_hot, batch_size=BATCH_SIZE, verbose=0)
_, accuracy_val = model.evaluate(X_test, Y_test_one_hot, batch_size=BATCH_SIZE, verbose=0)

# Plotando os resultados
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento', 'Validacao'])


print(f'--> Acuracia (treino): {accuracy_train:.4f}')
print(f'--> Acuracia (validacao): {accuracy_val:.4f}')
print(f"--> Ein - Eval = {accuracy_train - accuracy_val:.4f}")
print(f"--> val_loss - train_loss = {(history.history['val_loss'][-1] - history.history['loss'][-1]):.4f}\n")

In [ ]:
# Predição
Y_pred_train = np.round(model.predict(X_train))
print('Classification Report (treino):')
print(classification_report(Y_train_one_hot, Y_pred_train))

# Pegando a posição do maior valor
Y_pred_train = np.argmax(Y_pred_train, axis=1)

matriz_confusao = confusion_matrix(Y_train, Y_pred_train)
plt.imshow(matriz_confusao, cmap='Oranges')
plt.title('Matriz de confusão (treino)')
plt.colorbar()
tick_marks = np.arange(len(nomes_classes))

plt.xticks(tick_marks, nomes_classes, rotation=45)
plt.yticks(tick_marks, nomes_classes)
plt.ylabel('Classe verdadeira')
plt.xlabel('Classe prevista')
# Adicionando os valores numéricos dentro das células
for i in range(matriz_confusao.shape[0]):
    for j in range(matriz_confusao.shape[1]):
        plt.text(j, i, str(matriz_confusao[i, j]), horizontalalignment="center", color="black") # Adiciona o valor da célula
plt.show()

In [ ]:
# Predição
Y_pred_test = np.round(model.predict(X_test))
Y_test_one_hot = keras.utils.to_categorical(Y_test, 10)
print('Classification Report (teste):')
print(classification_report(Y_test_one_hot, Y_pred_test))

# Pegando a posição do maior valor
Y_pred_test = np.argmax(Y_pred_test, axis=1)

matriz_confusao = confusion_matrix(Y_test, Y_pred_test)
plt.imshow(matriz_confusao, cmap='Oranges')
plt.title('Matriz de confusão (teste)')
plt.colorbar()
tick_marks = np.arange(len(nomes_classes))

plt.xticks(tick_marks, nomes_classes, rotation=45)
plt.yticks(tick_marks, nomes_classes)
plt.ylabel('Classe verdadeira')
plt.xlabel('Classe prevista')
# Adicionando os valores numéricos dentro das células
for i in range(matriz_confusao.shape[0]):
    for j in range(matriz_confusao.shape[1]):
        plt.text(j, i, str(matriz_confusao[i, j]), horizontalalignment="center", color="black") # Adiciona o valor da célula
plt.show()